## Initialize the environment and get screener configuration

In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

from core.data_sources import CLOBDataSource
from core.features.candles.volatility import VolatilityConfig
from core.features.candles.volume import VolumeConfig

from research_notebooks.xtreet_bb.utils import generate_config, dump_dict_to_yaml, read_yaml_to_dict, fetch_candles, generate_screener_report

clob = CLOBDataSource()
screener_config_name = "okx_perpetual.yml"
screener_config_path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "screener_configs", screener_config_name)
screener_config = read_yaml_to_dict(screener_config_path)
screener_config

In [ ]:
# Screener parameters
CONNECTOR_NAME = screener_config["screener_params"]["connector_name"]
INTERVALS = screener_config["screener_params"]["intervals"]
DAYS = screener_config["screener_params"]["days_to_download"]
FETCH_CANDLES = screener_config["screener_params"]["download_candles"]
BATCH_CANDLES_REQUEST = screener_config["screener_params"]["batch_candles_request"]
SLEEP_REQUEST = screener_config["screener_params"]["sleep_request"]
FROM_TRADES = screener_config["screener_params"]["from_trades"]
VOLUME_THRESHOLD = screener_config["screener_params"]["volume_threshold"]  # From percentile VOLUME_THRESHOLD to 1
VOLATILITY_THRESHOLD = screener_config["screener_params"]["volatility_threshold"]  # From percentile VOLATILITY_THRESHOLD to 1
MAX_TOP_MARKETS = screener_config["screener_params"]["max_top_markets"]

# Trading Rules Filter
QUOTE_ASSET = screener_config["trading_rules_filter"]["quote_asset"]
MIN_NOTIONAL_SIZE = screener_config["trading_rules_filter"]["min_notional_size"]  # In USDT
MAX_PRICE_STEP = screener_config["trading_rules_filter"]["max_price_step"]  # Min price step in % (tick size)

VOLATILITY_WINDOW = screener_config["trading_rules_filter"]["volatility_window"]  # In bars
VOLUME_FAST_WINDOW = screener_config["trading_rules_filter"]["volume_fast_window"]  # No se usa
VOLUME_SLOW_WINDOW = screener_config["trading_rules_filter"]["volume_slow_window"]  # No se usa

# Config generation
TOTAL_AMOUNT = screener_config["config_generation"]["total_amount"]  # General total amount for all markets
ACTIVATION_BOUNDS = screener_config["config_generation"]["activation_bounds"]  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = screener_config["config_generation"]["max_executors_per_side"]  # Maximum number of executors per side
COOLDOWN_TIME = screener_config["config_generation"]["cooldown_time"]
LEVERAGE = screener_config["config_generation"]["leverage"] # Should be for each trading pair
TIME_LIMIT = screener_config["config_generation"]["time_limit"]
BOLLINGER_LENGTHS = screener_config["config_generation"]["bollinger_lengths"]
BOLLINGER_STDS = screener_config["config_generation"]["bollinger_stds"]
SL_STD_MULTIPLIER = screener_config["config_generation"]["sl_std_multiplier"]
TS_DELTA_MULTIPLIER = screener_config["config_generation"]["ts_delta_multiplier"]
MAX_DCA_AMOUNT_RATIO = screener_config["config_generation"]["max_dca_amount_ratio"] # Amount 2 / Amount 1


# Config filtering
MIN_DISTANCE_BETWEEN_ORDERS = screener_config["config_filtering"]["min_distance_between_orders"]
MAX_TS_SL_RATIO = screener_config["config_filtering"]["max_ts_sl_ratio"]

## Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)

trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

candles = await fetch_candles(download_candles=FETCH_CANDLES,
                              trading_pairs=trading_pairs,
                              batch_candles_request=BATCH_CANDLES_REQUEST,
                              clob=clob,
                              connector_name=CONNECTOR_NAME,
                              root_path=root_path,
                              intervals=INTERVALS,
                              days_to_download=DAYS,
                              sleep_request=SLEEP_REQUEST,
                              from_trades=FROM_TRADES)


## Generate screener report and filter out the top markets

In [ ]:
screener_report = generate_screener_report(
    candles=candles,
    trading_rules=trading_rules,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_FAST_WINDOW, long_window=VOLUME_FAST_WINDOW))

screener_report.sort_values("mean_natr", ascending=False, inplace=True)

# Calculate the 20th percentile (0.2 quantile) for both columns
natr_percentile = screener_report['mean_natr'].quantile(VOLATILITY_THRESHOLD)
volume_percentile = screener_report['average_volume_per_hour'].quantile(VOLUME_THRESHOLD)

# Filter the DataFrame to get observations where mean_natr is greater than its 20th percentile
# and average_volume_per_hour is greater than its 20th percentile
max_top_markets = min(MAX_TOP_MARKETS, screener_report.shape[0])
screener_top_markets = screener_report[
    (screener_report['mean_natr'] > natr_percentile) &
    (screener_report['average_volume_per_hour'] > volume_percentile) &
    (screener_report["price_step_pct"] < MAX_PRICE_STEP)
].sort_values(by="average_volume_per_hour").head(MAX_TOP_MARKETS)

# Display the filtered DataFrame
screener_top_markets[["trading_pair", "mean_natr", "average_volume_per_hour", "price_step_pct"]]

## Generate strategy configurations

In [ ]:
strategy_configs = generate_config(
    connector_name=CONNECTOR_NAME,
    intervals=INTERVALS,
    screener_top_markets=screener_top_markets,
    candles=candles,
    total_amount=TOTAL_AMOUNT,
    max_executors_per_side=MAX_EXECUTORS_PER_SIDE,
    cooldown_time=COOLDOWN_TIME,
    leverage=LEVERAGE,
    time_limit=TIME_LIMIT,
    bb_lengths=BOLLINGER_LENGTHS,
    bb_stds=BOLLINGER_STDS,
    sl_std_multiplier=SL_STD_MULTIPLIER,
    min_distance_between_orders=MIN_DISTANCE_BETWEEN_ORDERS,
    max_ts_sl_ratio=MAX_TS_SL_RATIO,
    ts_delta_multiplier=TS_DELTA_MULTIPLIER,
    max_dca_amount_ratio=MAX_DCA_AMOUNT_RATIO
)
n_trading_pairs = len(set([config["trading_pair"] for config in strategy_configs]))
n_configs = len(strategy_configs)
print(f"Generated {n_configs} configurations for {n_trading_pairs} trading pairs")

In [ ]:
for config in strategy_configs:
    dump_dict_to_yaml("configs/", config)